#Preprocessing

##import

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel('/content/ShopeeMain Bersih (1).xlsx')
df.head()

In [ ]:
df.info()

In [ ]:
df.drop_duplicates(subset='Review', keep='first', inplace=True)
df.info()

In [ ]:
df.reset_index(drop=True, inplace=True)
df.info()

In [ ]:
import pandas as pd
from datetime import datetime

# Memisahkan kolom Date menjadi Tanggal dan Waktu
df['Tanggal'] = df['Date'].dt.strftime('%d/%m/%Y')  # Format DD/MM/YYYY
df['Waktu'] = df['Date'].dt.strftime('%H:%M:%S')    # Format HH:MM:SS

# Menampilkan hasil
df.head()

In [ ]:
df = pd.DataFrame(df[['Tanggal','User','Review']])
df.head()

##cleaning

In [ ]:
import re
import string
import nltk

In [ ]:
import re

def remove_URL(tweet):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', tweet)

def remove_html(tweet):
    html = re.compile(r'<.*?>')
    return html.sub(r'', tweet)

def remove_emoji(tweet):
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002500-\U00002BEF"  # chinese char
                                u"\U00002702-\U000027B0"
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                u"\U0001f926-\U0001f937"
                                u"\U00010000-\U0010ffff"
                                u"\u200d"
                                u"\u2640-\u2642"
                                u"\u2600-\u2B55"
                                u"\u23cf"
                                u"\u23e9"
                                u"\u231a"
                                u"\ufe0f"  # dingbats
                                u"\u3030"
                                "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', tweet)

def remove_numbers(tweet):
    tweet = re.sub(r'\d+', '', tweet)
    return tweet

def remove_symbols(tweet):
    # Hanya menghapus simbol, mempertahankan huruf, angka, dan spasi
    tweet = re.sub(r'[^\w\s]', '', tweet)
    return tweet

def remove_extra_spaces(tweet):
    # Menghapus spasi berlebihan di antara kata-kata
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

# Proses cleaning data di DataFrame
df['cleaning'] = df['Review'].apply(lambda x: remove_URL(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_html(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_emoji(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_numbers(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_symbols(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_extra_spaces(x))

In [ ]:
df.head(5)

##pemisahan berdasarkan kapital

In [ ]:
def split_by_capital_letters(text):
    # Memisahkan kata yang digabung berdasarkan huruf kapital
    tokens = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?![a-z])', text)
    return ' '.join(tokens)  # Menggabungkan kembali token dengan spasi

# Terapkan fungsi split_by_capital_letters pada kolom 'cleansing'
df['capital_split'] = df['cleaning'].apply(split_by_capital_letters)
df.head(10)

##case folding

In [ ]:
def case_folding(text):
    if isinstance(text, str):
        return text.lower()  # Mengubah menjadi huruf kecil
    else:
        return text

# Terapkan fungsi case_folding ke kolom 'capital_split'
df['case_folding'] = df['capital_split'].apply(case_folding)

In [ ]:
df.head(5)

##normalisasi

In [ ]:
# Memuat data
kata_baku_df = pd.read_excel('/content/kamuskatabaku.xlsx')

# Mengubah kolom menjadi list
informal = kata_baku_df['tidak_baku'].values.tolist()
formal = kata_baku_df['kata_baku'].values.tolist()

# Fungsi formalisasi sudah didefinisikan sebelumnya
def formalize_text(text, formal, informal):
    text = text.split(" ")
    for i in range(len(text)):
        if text[i] in informal:
            index = informal.index(text[i])
            text[i] = formal[index]
    return ' '.join(map(str, text))

# Mengaplikasikan fungsi pada kolom 'case_folding'
df['formal'] = df['case_folding'].apply(lambda x: formalize_text(x, formal, informal))

In [ ]:
df

##tokenization

In [ ]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenization'] = df['formal'].apply(tokenize)
df.head(10)

##filtering/stopword removal

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

# Baca file stopword dari txt
with open('stopword.txt', 'r') as file:
    stopwords_list = file.readlines()

# Membersihkan karakter newline dan membuat set stopwords
custom_stopwords = set(word.strip() for word in stopwords_list)

# Fungsi untuk menghapus stopwords
def remove_stopwords_from_txt(tokens):
    return [word for word in tokens if word not in custom_stopwords]

# Contoh penerapan
df['filtering'] = df['tokenization'].apply(remove_stopwords_from_txt)


In [ ]:
df

###stopword nltk

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

In [ ]:
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

df['filtering'] = df['tokenization'].apply(remove_stopwords)
df.head(10)

##steaming

In [ ]:
!pip install Sastrawi

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
    return [stemmer.stem(word) for word in text]

df['Preprocessing'] = df['filtering'].apply(lambda x: ' '.join(stem_text(x)))
df.head(10)

In [ ]:
df.to_csv('Hasil Preprocessing.csv', encoding='utf8', index=False)

In [ ]:
df.to_excel('Hasil Preprocessing.xlsx', index=False)

#Visualisasi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud
from collections import Counter

# Combine all reviews with minimal preprocessing
text = ' '.join(df['Preprocessing'].fillna('').astype(str))

# WordCloud visualization
plt.figure(figsize=(10, 5))
wc = WordCloud(
    background_color="white",
    width=800,
    height=400,
    colormap='viridis'
)
wc.generate(text)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Reviews", pad=20, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Frequency distribution plot
plt.figure(figsize=(12, 6))
tokens = text.split()
word_counts = Counter(tokens)
top_words = word_counts.most_common(20)
word, count = zip(*top_words)

colors = plt.cm.viridis(np.linspace(0, 1, len(word)))
bars = plt.bar(word, count, color=colors)
plt.xlabel("Kata-Kata Sering Muncul", fontsize=12, fontweight='bold')
plt.ylabel("Jumlah Kata", fontsize=12, fontweight='bold')
plt.title("Frekuensi Kata", fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')

# Add frequency numbers on top of bars
for bar, num in zip(bars, count):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        num + 1,
        str(num),
        fontsize=10,
        ha='center'
    )

plt.tight_layout()
plt.show()

# Print top 20 words and their frequencies
print("\nTop 20 kata yang paling sering muncul:")
for word, freq in top_words:
    print(f"{word}: {freq}")

#Unsupervised Lexical

In [ ]:
!pip install pandas numpy Sastrawi openpyxl

In [ ]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Load lexicon data
# Pastikan path file sesuai dengan lokasi penyimpanan
lexicon_positive = pd.read_excel('/content/kamus_positif.xlsx')
lexicon_negative = pd.read_excel('/content/kamus_negatif.xlsx')

# Create dictionaries from lexicons
lexicon_positive_dict = dict(zip(lexicon_positive.iloc[:, 0], lexicon_positive.iloc[:, 1]))
lexicon_negative_dict = dict(zip(lexicon_negative.iloc[:, 0], lexicon_negative.iloc[:, 1]))

def sentiment_analysis_lexicon_indonesia(text):

    # Split text into words
    words = text.split()

    # Calculate sentiment score
    score = sum(lexicon_positive_dict.get(word, 0) for word in words) + \
           sum(lexicon_negative_dict.get(word, 0) for word in words)

    # Determine sentiment
    if score > 0:
        sentiment = 'positif'
    elif score < 0:
        sentiment = 'negatif'
    else:
        sentiment = 'netral'

    return score, sentiment

# Apply sentiment analysis to steaming column
results = df['Preprocessing'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))

# Add results to dataframe
df['Score'] = results[0]  # Sentiment score
df['Label'] = results[1]  # Sentiment label

# Display specific columns
selected_columns = df[['Tanggal', 'User', 'Review', 'Preprocessing', 'Label']]
print(selected_columns.head())


In [ ]:
df = pd.DataFrame(df[['Tanggal','User','Review','Preprocessing','Label']])
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Create figure with two subplots
plt.figure(figsize=(15, 8))

# Bar Chart
plt.subplot(1, 2, 1)
sentiment_counts = df['Label'].value_counts()
colors = ['#2ecc71', '#e74c3c', '#3498db']  # green for positive, red for negative, blue for neutral
bars = plt.bar(sentiment_counts.index, sentiment_counts.values, color=colors)
plt.title('Distribusi Label Sentimen', pad=20, fontsize=14, fontweight='bold')
plt.xlabel('Label Sentimen', fontsize=12)
plt.ylabel('Jumlah', fontsize=12)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}\n({height/len(df):.1%})',
             ha='center', va='bottom')

# Pie Chart
plt.subplot(1, 2, 2)
plt.pie(sentiment_counts.values,
        labels=sentiment_counts.index,
        colors=colors,
        autopct='%1.1f%%',
        explode=(0.05, 0.05, 0.05),  # Give some separation to all pieces
        shadow=True,
        startangle=90)
plt.title('Persentase Label Sentimen', pad=20, fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print numerical summary
print("\nRingkasan Analisis Sentimen:")
summary_df = pd.DataFrame({
    'Jumlah': sentiment_counts,
    'Persentase': (sentiment_counts / len(df) * 100).round(2)
})
print(summary_df)

In [ ]:
!pip install tabulate

In [ ]:
from tabulate import tabulate

# Filter review negatif
negative_reviews = df[df['Label'] == 'negatif'][['Tanggal', 'User', 'Review', 'Preprocessing', 'Label']]

# Reset index agar nomor baris dimulai dari 1
negative_reviews = negative_reviews.reset_index(drop=True)

# Membuat tabel dengan tabulate
print("\nReview dengan sentimen negatif:")
print(tabulate(negative_reviews, headers='keys', tablefmt='grid', showindex=True))

# Menampilkan jumlah total review negatif
print(f"\nTotal review negatif: {len(negative_reviews)}")

##visualisasi

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Pisahkan kata-kata pada setiap label
def count_words_by_label(df, label):
    # Filter dataframe berdasarkan label
    label_df = df[df['Label'] == label]

    # Gabungkan semua teks pada kolom 'Preprocessing'
    all_text = ' '.join(label_df['Preprocessing'])

    # Hitung frekuensi kata
    word_counts = Counter(all_text.split())
    return word_counts

# Hitung frekuensi kata untuk setiap label
positif_counts = count_words_by_label(df, 'positif')
negatif_counts = count_words_by_label(df, 'negatif')
netral_counts = count_words_by_label(df, 'netral')

# Gabungkan frekuensi kata ke dalam dataframe
top_n = 10  # Tampilkan hanya 10 kata teratas
freq_df = pd.DataFrame({
    'positif': dict(positif_counts.most_common(top_n)),
    'negatif': dict(negatif_counts.most_common(top_n)),
    'netral': dict(netral_counts.most_common(top_n))
}).fillna(0).astype(int)

# Plot barchart
freq_df.plot(kind='bar', figsize=(12, 6))
plt.title('Frekuensi Kata pada Setiap Label')
plt.ylabel('Frekuensi')
plt.xlabel('Kata')
plt.xticks(rotation=45)
plt.legend(title='Label')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

# Fungsi untuk menghitung frekuensi kata per label
def get_word_frequency_by_label(df, label):
    # Mengambil semua teks dari label tertentu
    texts = df[df['Label'] == label]['Preprocessing']

    # Menggabungkan semua kata
    all_words = ' '.join(texts).split()

    # Menghitung frekuensi
    word_freq = Counter(all_words)

    # Mengambil 20 kata teratas
    top_20 = pd.DataFrame(word_freq.most_common(20), columns=['Kata', 'Frekuensi'])

    return top_20

# Membuat visualisasi untuk setiap label
def plot_word_frequency(df, label):
    plt.figure(figsize=(12, 6))
    data = get_word_frequency_by_label(df, label)

    # Membuat bar chart
    sns.barplot(x='Frekuensi', y='Kata', data=data, palette='viridis')

    plt.title(f'20 Kata Terbanyak untuk Label {label.title()}')
    plt.xlabel('Frekuensi')
    plt.ylabel('Kata')
    plt.tight_layout()
    plt.show()

# Menjalankan analisis untuk setiap label
labels = ['positif', 'negatif', 'netral']

for label in labels:
    # Menampilkan data frekuensi
    print(f"\nFrekuensi kata untuk label {label}:")
    freq_data = get_word_frequency_by_label(df, label)
    print(freq_data)

    # Menampilkan visualisasi
    plot_word_frequency(df, label)